## Análisis de Simulated Annealing para una fuerza de ventas de 6 nodos

Simulated Annhealing es un algoritmo que recibe los siguientes hiperárametros de entrada: 

+ Tmax
+ Tmin 
+ steps 
+ updates 

Los resultados obtenidos pueden verse afectados al variar los valores de tales hiperparámetros. Por otro lado, dependiendo del número de nodos del grafo, estos hiperparámetros también podrían afectar la ruta mínima encontrada por el algoritmo.    

El presente notebook considera la implementación de simulated annhealing para un grafo con 6 nodos. Como primero objetivo se variarán los hiperparámetros y se identificarán aquellos que den mejores resultados. Entendiéndose como mejores resultados; obtener la ruta con menor distancia. Adicionalmente, una vez seleccionados los mejores hiperparámetros, se correrá el algoritmo 10 veces con el fin de realizar un análisis sobre las rutas obtenidas. Particularmente se tiene interés en revisar variaciones en las rutas obtenidas en cada corrida.

Dentro del conjunto de datos que se tienen disponibles, existen varias fuerzas de venta que deben recorrer 6 nodos. Se decidió elegir la fuerza de venta **80993** perteneciente al estado de Nuevo León para llevar a cabo estas pruebas.

In [1]:
!pip install dynaconf
!pip install psycopg2-binary
!pip install simanneal

In [2]:
# Librerías

import pandas as pd
import sys
sys.path.append('../../')
%load_ext autoreload
%autoreload 2

from src import Utileria as ut
from src.models import particle_swarm as ps
from src.models import simulated_annealing as sa

### 1. Búsqueda de mejores hiperparámetros

**1.1 Definición de datos de entrada**
+ Grafo completo de los puntos que debe vistar la fuerza de ventas
+ Hiperámetros con los que correrá el algoritmo

In [3]:
# Se obtiene el dataframe que contiene el grafo de la fuerza de venta a evaluar:
str_Query = 'select id_origen, id_destino, distancia from trabajo.grafos where id_fza_ventas={};'

# En el query se especifica el id_fza_venta del cual se quiere obtener su grafo
df_Grafo = ut.get_data(str_Query.format(80993))
df_Grafo

,id_origen,id_destino,distancia
0,11037,1001402004,0.267770992210818
1,11037,1006681965,0.3250713086280114
2,11037,1020053072,2.3599263518563016
3,11037,1020235635,1.2662382942527688
4,11037,1020402992,0.07674597387253217
5,1001402004,1006681965,0.4982193543875845
6,1001402004,1020053072,2.611292644038669
7,1001402004,1020235635,1.5321611628202725
8,1001402004,1020402992,0.23296305988637042
9,1006681965,1020053072,2.141257356651251


In [4]:
# Se crea el diccionario de hiper-parámetros que se evaluarán

#Default parameters
#Tmax = 25000.0  # Max (starting) temperature
#Tmin = 2.5      # Min (ending) temperature
#steps = 50000   # Number of iterations
#updates = 100   # Number of updates (by default an update prints to stdout)

dict_Hiper_SA = {'tmax': {10000, 25000},
              'tmin': {1,2.5,5},
              'steps': {500,5000},
              'updates': {10,50,100}
              }



**1.2 Gridsearch**

Dentro de la clase Utileria fue definido un método llamado *GridSearch*, el cual recibe como parámetros el grafo de una fuerza de ventas fijo, un diccionario de parámetros, el algoritmo a evaluar y el número de iteraciones que se correrá por cada combinación de hiperámetros. Este método evalúa el algoritmo con todas las combinaciones que se pueden generar a partir del diccionario de parámetros. 

En este caso se considerarán 3 valores de *updates*, 2 valores del *steps*, 2 valores de *Tmax* y 2 de *Tmin* dando lugar a un total de 36 combinaciones. 

Cada combinación de hiperarámetros se correrá 100 veces y como resultado se obtendrá una tabla indicando los Hiperámetros utilizados, las distancias mínima y máxima obtenidas dentro de las 100 corridas; y el número de corridas en que se repitió tal distancia mínima.

In [5]:
%%time

# Se corre el GridSearch para el grafo y los hiperparámetros previamente definidos

df_Resultado = ut.GridSearch(df_Grafo, sa.SimulatedAnnealing, dict_Hiper_SA, 100)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.24    68.00%    26.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.14    68.00%    24.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.77    76.00%    24.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.81    66.00%    16.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.70    82.00%    32.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.11    62.00%    18.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          7.94    68.00%    28.00%     0:00:00     0:00:00 Temperature 

     1.00000          5.96    72.00%    26.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          7.52    80.00%    32.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.14    80.00%    36.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          7.66    74.00%    24.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.68    66.00%    24.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          8.00    78.00%    26.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.66    78.00%    36.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000 

     1.00000          5.77    75.40%    28.20%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.60    81.20%    30.40%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.77    81.60%    31.60%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.85    78.00%    31.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          7.88    79.40%    31.80%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          8.18    81.60%    28.20%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.85    78.00%    31.20%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000 

     1.00000          8.44    78.80%    31.20%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          7.88    81.20%    35.40%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.23    78.40%    32.40%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.87    81.20%    31.60%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.70    81.60%    31.60%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.24    82.00%    34.20%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.22    81.40%    33.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000 

     1.00000          5.96    79.00%    34.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.70    79.00%    30.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.85    73.00%    24.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.22    70.00%    30.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.26    68.00%    25.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.24    73.00%    30.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.24    81.00%    31.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000 

...

     5.00000          6.30    98.00%    40.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          6.22    98.00%    38.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          7.90    96.00%    40.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          6.30   100.00%    44.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          7.99    98.00%    42.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          5.86    94.00%    36.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          6.23    98.00%    40.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000 

     5.00000          8.03    96.00%    38.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          8.53    98.00%    42.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          8.18    98.00%    36.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          7.94    94.00%    40.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          8.56    94.00%    40.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          5.87    98.00%    36.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          8.14   100.00%    40.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000 

     5.00000          6.26    80.00%    30.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          6.26    70.00%    20.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          5.70    90.00%    60.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          7.99    90.00%    20.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          5.85    70.00%    20.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          7.61   100.00%    30.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          7.54    90.00%    30.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000 

     5.00000          7.52    90.00%    30.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          5.60   100.00%    60.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          7.56   100.00%    40.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     5.00000          6.14   100.00%    30.00%     0:00:00     0:00:00

CPU times: user 15min, sys: 48.2 s, total: 15min 48s
Wall time: 15min 55s


In [6]:
# Se muestra el dataframe con los resultados obtenidos de la corrida del GridSearch
pd.options.display.max_colwidth = 100
df_Resultado

,HiperParámetros,Distancia mínima (km),Distancia máxima (km),Frec. rel. dist. min.
0,"{'tmax': 10000, 'tmin': 1.0, 'steps': 5000, 'updates': 100}",5.604,5.604,100/100
1,"{'tmax': 10000, 'tmin': 1.0, 'steps': 5000, 'updates': 10}",5.604,5.604,100/100
2,"{'tmax': 10000, 'tmin': 1.0, 'steps': 5000, 'updates': 50}",5.604,5.604,100/100
3,"{'tmax': 10000, 'tmin': 1.0, 'steps': 500, 'updates': 100}",5.604,5.604,40/100
4,"{'tmax': 10000, 'tmin': 1.0, 'steps': 500, 'updates': 10}",5.604,5.604,36/100
5,"{'tmax': 10000, 'tmin': 1.0, 'steps': 500, 'updates': 50}",5.604,5.604,43/100
6,"{'tmax': 10000, 'tmin': 2.5, 'steps': 5000, 'updates': 100}",5.604,5.604,99/100
7,"{'tmax': 10000, 'tmin': 2.5, 'steps': 5000, 'updates': 10}",5.604,5.604,99/100
8,"{'tmax': 10000, 'tmin': 2.5, 'steps': 5000, 'updates': 50}",5.604,5.604,99/100
9,"{'tmax': 10000, 'tmin': 2.5, 'steps': 500, 'updates': 100}",5.604,5.604,36/100


**1.3 Análisis y Resultados**

En primer lugar es importante mencionar las motivaciones de ciertos de los hiperparámetros elegidos. 

+ Con respecto al número de iteraciones,se escogieron $100$ iteraciones para poder ver la variación entre iteraciones y la estabilidad del algoritmo. El número fue suficiente ya que en ninguna combinación de parámetros hubo variación en la distancia minima. 

+ Sobre el resto de los parámetros, se escogieron valores alrededor e incluyendo los parámetros por omisión del algoritmo. 

Para poder interpretar los resultados mostrados en el dataframe anterior es conveniente recordar que se realizaron 36 pruebas, es decir, se tuvieron 36 combinaciones de hiperparámetros; y cada una de esas combinaciones se corrió 100 veces, dando un total de 360 corridas. En las 100 corridas de cada prueba se registró la distancia mínima obtenida, la distancia máxima y la frecuencia relativa de la distancia mínima. A continuación se enlistan observaciones importantes derivadas de estas pruebas:

+ La distancia mínima y máxima obtenida: $5.6 km$ es la misma para las 36 combinaciones. Esto demuestra que apesar de utilizar los mismos hiperparámetros, el algoritmo da los mismos resultados. 
 
+ Estos resultados pueden ser consecuencia de haber utilizando los parámetros por default, por eso se hizo el siguiente experimento: correr el algoritmo 100 veces solo con 1 paso, updates con valor 2. Tmin con valor 1 y Tmax con valor 2. Fue interesante que la distancia máxima en este caso ya fue más de 8 km.


In [7]:
%%time

# Se corre el GridSearch para el grafo y los hiperparámetros previamente definidos
dict_Hiper_SA = {'tmax': {2}, 'tmin': {1}, 'steps': {1}, 'updates': {2}}

df_Resultado = ut.GridSearch(df_Grafo, sa.SimulatedAnnealing, dict_Hiper_SA, 100)
pd.options.display.max_colwidth = 100
df_Resultado

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.85   100.00%   100.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          7.61   100.00%   100.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.26   100.00%   100.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          7.88     0.00%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.30   100.00%   100.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.68     0.00%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          7.91   100.00%     0.00%     0:00:00     0:00:00 Temperature 

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          8.09   100.00%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.34   100.00%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          7.88   100.00%   100.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.77     0.00%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          8.03   100.00%     0.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          8.05   100.00%   100.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.68   100.00%   100.00%     0:00:00     0:00:00 Temperature 

CPU times: user 14.8 s, sys: 301 ms, total: 15.1 s
Wall time: 14.8 s


,HiperParámetros,Distancia mínima (km),Distancia máxima (km),Frec. rel. dist. min.
0,"{'tmax': 2, 'tmin': 1, 'steps': 1, 'updates': 2}",5.604,8.532,1/100


### 2. GridSearch con parametros simples

In [8]:
rutas = pd.DataFrame(index=range(100),columns=['km', 'Ruta'])


dict_Hiper_SA = {'tmax': 10000, 'tmin': 1.0, 'steps': 5000, 'updates': 100}
              



In [9]:
rutas

,km,Ruta
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
95,NaN,NaN
96,NaN,NaN
97,NaN,NaN
98,NaN,NaN


In [10]:
for corrida in range(100):
    SA = sa.SimulatedAnnealing(df_Grafo,dict_Hiper_SA)
    SA.Ejecutar()
    
    min_distancia = round(SA.nbr_MejorCosto,3)
    mejor_ruta =  ut.convert(SA.lst_MejorCamino)
    
    rutas.km[corrida] = min_distancia
    rutas.Ruta[corrida] = mejor_ruta
    
    

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.70    84.00%    30.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          7.56    68.00%    32.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.81    68.00%    26.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.60    68.00%    24.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.86    66.00%    16.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.70    80.00%    30.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.26    68.00%    18.00%     0:00:00     0:00:00 Temperature 

     1.00000          8.07    80.00%    36.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.77    72.00%    20.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.85    62.00%    28.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          6.24    62.00%    26.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.96    68.00%    22.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.70    70.00%    24.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000          5.68    86.00%    34.00%     0:00:00     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     1.00000 

In [11]:
# Primeras 7 rutas obtenidas
rutas.head(7)

,km,Ruta
0,5.604,11037-1020235635-1020053072-1006681965-1001402004-1020402992
1,5.604,11037-1020235635-1020053072-1006681965-1001402004-1020402992
2,5.604,11037-1020235635-1020053072-1006681965-1001402004-1020402992
3,5.604,11037-1020235635-1020053072-1006681965-1001402004-1020402992
4,5.604,11037-1020235635-1020053072-1006681965-1001402004-1020402992
5,5.604,11037-1020235635-1020053072-1006681965-1001402004-1020402992
6,5.604,11037-1020235635-1020053072-1006681965-1001402004-1020402992


In [12]:
# Descripción de las rutas obtenidas

rutas.groupby('Ruta').describe()

km         \
                                                              count unique   
Ruta                                                                         
11037-1020235635-1020053072-1006681965-1001402004-1020402992  100.0    1.0   

                                                                            
                                                                top   freq  
Ruta                                                                        
11037-1020235635-1020053072-1006681965-1001402004-1020402992  5.604  100.0

### Conclusiones:
+ Los resultados del algoritmo son estables.